In [71]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import pandas as pd

In [72]:
meta_df = pd.read_csv('./data/archive/HAM10000_metadata.csv', delimiter=',')

meta_df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [73]:
data_df = pd.read_csv('./data/archive/hmnist_28_28_RGB.csv', delimiter=',')

In [74]:
data_df = pd.read_csv('./data/archive/hmnist_28_28_RGB.csv')

X = data_df.drop('label', axis=1)
y = data_df['label']

In [75]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [76]:
# X_train = X_train.values
# X_test = X_test.values

# # Reshape the data
# img_size = int(np.sqrt(X_train.shape[1] // 3))  # Assuming square RGB images
# X_train = X_train.reshape(-1, img_size, img_size, 3)
# X_test = X_test.reshape(-1, img_size, img_size, 3)

# total_pixels = X_train.shape[1]
# img_size = np.sqrt(total_pixels // 3)

# # Normalize the data
# X_train = X_train.astype('float32') / 255.0
# X_test = X_test.astype('float32') / 255.0

In [81]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.image import smart_resize

# Convert to numpy arrays if not already
X_train = X_train.values if hasattr(X_train, 'values') else X_train
X_test = X_test.values if hasattr(X_test, 'values') else X_test

# Function to reshape and resize images
def reshape_and_resize(data, target_size=(32, 32)):
    # Calculate the original image size
    img_size = int(np.sqrt(data.shape[1] // 3))  # Assuming square RGB images

    # Reshape the data to 3D images
    reshaped_data = data.reshape(-1, img_size, img_size, 3)

    # Resize images to target size (32x32)
    resized_data = np.array([smart_resize(img, target_size) for img in reshaped_data])

    return resized_data

# Reshape and resize the data
X_train = reshape_and_resize(X_train)
X_test = reshape_and_resize(X_test)

# Normalize the data
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

print("Training data shape:", X_train.shape)
print("Testing data shape:", X_test.shape)

# Now X_train and X_test are ready to be used with your model

Training data shape: (8012, 32, 32, 3)
Testing data shape: (2003, 32, 32, 3)


In [78]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.1
)

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weight_dict = dict(enumerate(class_weights))

In [79]:
# import tensorflow as tf
# from tensorflow.keras import layers, models, optimizers
# from tensorflow.keras.applications import ResNet50
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report, confusion_matrix
# import numpy as np

# # Squeeze and Excitation block
# def squeeze_excite_block(input_tensor, ratio=16):
#     channels = input_tensor.shape[-1]
#     se = layers.GlobalAveragePooling2D()(input_tensor)
#     se = layers.Reshape((1, 1, channels))(se)
#     se = layers.Dense(channels // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
#     se = layers.Dense(channels, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)
#     return layers.multiply([input_tensor, se])

# # SE-ResNeXt block
# def se_resnext_block(input_tensor, filters, cardinality=32):
#     residual = input_tensor

#     x = layers.Conv2D(filters, kernel_size=1, use_bias=False)(input_tensor)
#     x = layers.BatchNormalization()(x)
#     x = layers.Activation('relu')(x)

#     x = layers.Conv2D(filters, kernel_size=3, padding='same', groups=cardinality, use_bias=False)(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.Activation('relu')(x)

#     x = layers.Conv2D(filters * 2, kernel_size=1, use_bias=False)(x)
#     x = layers.BatchNormalization()(x)

#     x = squeeze_excite_block(x)

#     if input_tensor.shape[-1] != filters * 2:
#         residual = layers.Conv2D(filters * 2, kernel_size=1, strides=1, use_bias=False)(input_tensor)
#         residual = layers.BatchNormalization()(residual)

#     x = layers.add([x, residual])
#     x = layers.Activation('relu')(x)

#     return x

# # SE-ResNeXt-50 model
# def se_resnext50(input_shape, num_classes):
#     inputs = layers.Input(shape=input_shape)

#     x = layers.Conv2D(64, kernel_size=7, strides=2, padding='same', use_bias=False)(inputs)
#     x = layers.BatchNormalization()(x)
#     x = layers.Activation('relu')(x)
#     x = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(x)

#     x = se_resnext_block(x, 64)
#     x = se_resnext_block(x, 64)
#     x = se_resnext_block(x, 64)

#     x = se_resnext_block(x, 128)
#     x = se_resnext_block(x, 128)
#     x = se_resnext_block(x, 128)
#     x = se_resnext_block(x, 128)

#     x = se_resnext_block(x, 256)
#     x = se_resnext_block(x, 256)
#     x = se_resnext_block(x, 256)
#     x = se_resnext_block(x, 256)
#     x = se_resnext_block(x, 256)
#     x = se_resnext_block(x, 256)

#     x = se_resnext_block(x, 512)
#     x = se_resnext_block(x, 512)
#     x = se_resnext_block(x, 512)

#     x = layers.GlobalAveragePooling2D()(x)
#     x = layers.Dense(num_classes, activation='softmax')(x)

#     model = models.Model(inputs, x)
#     return model

# # Create and compile the model
# input_shape = X_train.shape[1:]  # (height, width, channels)
# num_classes = len(np.unique(y))
# model = se_resnext50(input_shape, num_classes)

# model.compile(
#     optimizer=optimizers.Adam(learning_rate=0.001),
#     loss='sparse_categorical_crossentropy',
#     metrics=['accuracy']
# )

# # Train the model
# history = model.fit(
#     datagen.flow(X_train, y_train, batch_size=32),
#     steps_per_epoch=len(X_train) // 32,
#     epochs=50,
#     validation_data=(X_test, y_test),
#     class_weight=class_weight_dict,
#     learning_rate=0.001,
#     callbacks=[
#         tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
#         tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
#     ]
# )

# # Evaluate the model
# y_pred = model.predict(X_test)
# y_pred_classes = np.argmax(y_pred, axis=1)

# print(classification_report(y_test, y_pred_classes))
# print(confusion_matrix(y_test, y_pred_classes))

# # Save the model
# model.save('se_resnext50_melanoma_model.h5')

In [82]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import ResNet50

class MultiHeadSelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0:
            raise ValueError(f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}")
        self.projection_dim = embed_dim // num_heads
        self.query_dense = layers.Dense(embed_dim)
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        self.combine_heads = layers.Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)
        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)

        attention = self.attention(query, key, value)
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(attention, (batch_size, -1, self.embed_dim))
        output = self.combine_heads(concat_attention)
        return output

def cnn_transformer_model(input_shape, num_classes):
    # CNN backbone (using ResNet50 as an example)
    cnn_model = ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)

    # Freeze the CNN layers (optional, depending on your dataset size and task)
    for layer in cnn_model.layers:
        layer.trainable = False

    inputs = cnn_model.input
    x = cnn_model.output

    # Reshape the output for self-attention
    _, h, w, c = x.shape
    x = layers.Reshape((h * w, c))(x)

    # Apply self-attention
    x = MultiHeadSelfAttention(embed_dim=c, num_heads=8)(x)

    # Global average pooling
    x = layers.GlobalAveragePooling1D()(x)

    # Final classification layer
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    return model

# Create and compile the model
input_shape = X_train.shape[1:]  # (height, width, channels)
num_classes = len(np.unique(y))
model = cnn_transformer_model(input_shape, num_classes)

model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    steps_per_epoch=len(X_train) // 32,
    epochs=50,
    validation_data=(X_test, y_test),
    class_weight=class_weight_dict,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
    ]
)

# Evaluate the model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred_classes))
print(confusion_matrix(y_test, y_pred_classes))

# Save the model
model.save('cnn_transformer_melanoma_model.h5')

Epoch 1/50


/Users/robpickerill/scm/personal/github.com/robpickerill/thesis/.venv/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


250/250 ━━━━━━━━━━━━━━━━━━━━ 57s 214ms/step - accuracy: 0.1473 - loss: 14.5781 - val_accuracy: 0.6026 - val_loss: 1.5100 - learning_rate: 0.0010
Epoch 2/50
  1/250 ━━━━━━━━━━━━━━━━━━━━ 54s 217ms/step - accuracy: 0.5000 - loss: 2.1744

/nix/store/bba5wwp7znmwdn90i3lhqm3cm2vkq64l-python3-3.11.9/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.5000 - loss: 2.1744 - val_accuracy: 0.2676 - val_loss: 1.6640 - learning_rate: 0.0010
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 52s 207ms/step - accuracy: 0.2029 - loss: 2.1508 - val_accuracy: 0.1722 - val_loss: 1.8254 - learning_rate: 0.0010
Epoch 4/50
  1/250 ━━━━━━━━━━━━━━━━━━━━ 46s 186ms/step - accuracy: 0.1562 - loss: 1.3693

2024-09-08 19:22:33.961662: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.1562 - loss: 1.3693 - val_accuracy: 0.1757 - val_loss: 1.7674 - learning_rate: 0.0010
Epoch 5/50
212/250 ━━━━━━━━━━━━━━━━━━━━ 7s 191ms/step - accuracy: 0.1889 - loss: 2.1284

KeyboardInterrupt: 